## Section 1: Problem Description

### 1. Problem Statement
This project builds an autoencoder for image reconstruction on the CIFAR-10 dataset.  
An autoencoder learns to compress images and then reconstruct them back.  
The main goal is to implement and speed up this process using CUDA on GPU, because CPU training is very slow for neural networks.

### 2. CIFAR-10 Dataset Overview
CIFAR-10 is a popular image dataset for computer vision tasks.

- Total images: 60,000  
- Image size: 32 × 32 pixels  
- Color channels: 3 (RGB)  
- Classes (10): airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck  
- Training set: 50,000 images  
- Test set: 10,000 images  

Each image is stored as unsigned 8-bit values.

**Data preprocessing**:
- Pixel values are normalized from \[0, 255\] to \[0, 1\] by dividing by 255.
- Labels are ignored during autoencoder training.
- No data augmentation is applied.

(Sample images from each class will be shown here.)

### 3. Autoencoder Architecture
The autoencoder has two main parts: an encoder and a decoder.  
The encoder compresses the image into a smaller representation.  
The decoder reconstructs the image from that representation.

**Input size**: 32 × 32 × 3  
**Latent size**: 8 × 8 × 128 = 8,192 features  
**Output size**: 32 × 32 × 3  

#### Encoder
- Conv2D: 3 → 256 channels, kernel 3×3, padding 1  
- ReLU  
- MaxPool2D: 2×2 → output 16×16×256  
- Conv2D: 256 → 128 channels, kernel 3×3, padding 1  
- ReLU  
- MaxPool2D: 2×2 → output 8×8×128  
![Encoder](resources/Encoder.png)

#### Decoder
- Conv2D: 128 → 128 channels, kernel 3×3, padding 1  
- ReLU  
- UpSample2D: 2×2 → output 16×16×128  
- Conv2D: 128 → 256 channels, kernel 3×3, padding 1  
- ReLU  
- UpSample2D: 2×2 → output 32×32×256  
- Conv2D: 256 → 3 channels, kernel 3×3, padding 1  
- No activation function in the last layer  

The decoder mirrors the encoder structure to help image reconstruction.
![Decoder](resources/Decoder.png)


### 4. Project Objectives
- **Performance**: Achieve large speedup using GPU compared to CPU (target >20×).  
- **Learning**: Understand autoencoders, CUDA programming, and GPU optimization.  
- **Quality**: Reconstruct CIFAR-10 images with low reconstruction loss.  
- **Pipeline**: Use the trained encoder to extract features for later classification.


## Section 2: Implementation Phases

### Phase 2.1: CPU Baseline Implementation

#### Objectives
- Build a correct autoencoder running on CPU.
- Verify forward and backward passes work as expected.
- Measure time and loss as a baseline before GPU optimization.
- This phase is required to ensure correctness before moving to CUDA.

#### Implementation Details

##### Data Pipeline
- Load CIFAR-10 data from binary files.
- Each image consists of 1 label byte and 3072 image bytes (32×32×3).
- Normalize pixel values from [0, 255] to [0, 1].
- **Training uses only 1,000 images (~2% of CIFAR-10 training set)** to speed up CPU experiments.
- **Testing uses the full 10,000-image test set**.

##### Layer Implementations
- **Conv2D**: 3×3 convolution with padding, implemented using nested CPU loops.
- **ReLU**: Element-wise max(0, x).
- **MaxPool2D**: 2×2 pooling with stride 2, take maximum value.
- **UpSample2D**: Nearest-neighbor upsampling by factor 2.

- **Forward pass**:  
  Each layer first calls the `forward()` function of its previous layer.  
  After the previous output is ready, the current layer computes its own output.

- **Backward pass**:  
  The execution order is reversed.  
  Each layer calls the `backward()` function of the next layer first, then computes its own gradients.

This design keeps the layer connections simple and makes debugging easier.

##### Training Loop
- Loop over epochs.
- Shuffle the 1,000 training images each epoch.
- For each image:
  - Forward pass through encoder and decoder.
  - Compute MSE loss.
  - Backward pass.
  - Update weights using SGD.
- Save model parameters after each epoch.

##### Key Code Snippets

Convolution function signature:
```
void convolve_cpu(
    float *dst,
    const float *src,
    const float *kernel,
    int col,
    int row,
    int kernel_width
);
```

Main training loop:
```
for (const auto& image : image_refs) {
    input->setImage(image->data);
    output->forward();
    output->backward(learning_rate, nullptr);
}
```

#### Results

- **Training data**: 1,000 images (≈2% of CIFAR-10 training set).
- **Test data**: Full 10,000-image CIFAR-10 test set.
- **Reconstruction loss**:
  - Training MSE loss ≈ **0.01**.
  - Test MSE loss ≈ **0.01**.
- This result is **surprising**, because the model was trained on a very small subset but still shows similar loss on the full test set.
- Loss values are stable during evaluation.
- Reconstructed images preserve overall structure but are blurry.
- **Performance (CPU, 2% dataset only)**:
  - Average epoch time: **59.35 seconds**
  - Total training time: **11,869.94 seconds**
- Total parameters: ~751,875, small enough for CPU memory.

![CPU_comparison](resources/CPU_comparison.png)
#### Key Takeaways
- Even training on only 1,000 images, the autoencoder generalizes well in terms of MSE.
- CPU performance is very slow, mainly due to convolution.
- Conv2D is the main bottleneck.
- These observations strongly motivate moving convolution and training to GPU.

### Phase 2.5: SVM Integration

#### Objectives
- Use the trained encoder to extract image features.
- Train an SVM classifier on these features.
- Evaluate the full image classification pipeline.

#### Implementation Details

##### Feature Extraction
- Only the **encoder** part of the autoencoder is used.
- For each image, a forward pass is executed.
- The encoder output is taken as the feature vector.
- Feature size is **8 × 8 × 128 = 8192 dimensions**.
- Features are extracted for:
  - 50,000 training images
  - 10,000 test images
- Features and labels are saved into CSV files for later use.

Feature extraction logic:
```
input->setImage(image.data);
(*layers.rbegin())->forward();

const float* enc_dev = encoder_layer->output();
cudaMemcpy(enc_host.data(), enc_dev,
           feature_size * sizeof(float),
           cudaMemcpyDeviceToHost);
```

##### SVM Integration
- Extracted features are loaded using cuDF.
- Features are normalized using `StandardScaler`.
- SVM is trained using cuML `SVC`, which runs on GPU.
- This avoids implementing SVM from scratch and is fast.

##### Hyperparameter Selection
- Kernel: RBF
- C = 10.0
- gamma = "scale"
- These values give good accuracy without long training time.

SVM training code:
```
model = SVC(kernel='rbf', C=10.0, gamma='scale')
model.fit(X_train, y_train)
```

#### Results

- **Feature extraction**:
  - Training set: 50,000 images
  - Test set: 10,000 images
  - Feature size: 8192 per image
- **SVM training time**: ~46 seconds
- **Classification accuracy**:
  - Training accuracy: **86.03%**
  - Test accuracy: **67.56%**

##### Confusion Matrix Summary
- Vehicles (ship, car, truck, plane) are classified very well.
- Animals (cat, dog, bird) have lower accuracy.
- Strong confusion exists between similar animals, especially cat and dog.
![Confusion Matrix](resources/ConfusionMatrix.png)
#### Analysis
- **Easiest classes**: ship, frog, plane, car.
- **Hardest classes**: cat, bird, dog.
- The confusion matrix shows most errors are between visually similar classes.
- Test accuracy is slightly higher than the expected range (60–65%).

#### Key Takeaways
- The encoder learns meaningful and reusable features.
- The two-stage approach (autoencoder + SVM) works well.
- GPU-based feature extraction and SVM give good end-to-end performance.
